Download the text--Peter Pan, remove the gutenberg_ig column and remove rows that are empty.

In [2]:
import re
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

pandas2ri.activate()
importr('gutenbergr')

df = ro.r('gutenberg_download("16")')
df.drop('gutenberg_id', inplace=True, axis=1)
df = df[df['text'] != ''].reset_index(drop=True)

In [3]:
df.iloc[:15]['text']

0                                             PETER PAN
1                                     [PETER AND WENDY]
2                By J. M. Barrie [James Matthew Barrie]
3     A Millennium Fulcrum Edition (c)1991 by Duncan...
4                                             Contents:
5                        Chapter 1 PETER BREAKS THROUGH
6                                  Chapter 2 THE SHADOW
7                       Chapter 3 COME AWAY, COME AWAY!
8                                  Chapter 4 THE FLIGHT
9                        Chapter 5 THE ISLAND COME TRUE
10                           Chapter 6 THE LITTLE HOUSE
11                  Chapter 7 THE HOME UNDER THE GROUND
12                       Chapter 8 THE MERMAID'S LAGOON
13                             Chapter 9 THE NEVER BIRD
14                            Chapter 10 THE HAPPY HOME
Name: text, dtype: object

Capture the title

In [4]:
title = df.iloc[0]['text']
title

'PETER PAN'

Capture the author

In [11]:
author = df.iloc[2]['text']
author = re.search('^By.*?\[(\w+ \w+ \w+)\]',author).group(1)
author

'James Matthew Barrie'

See how many occurances of Chapter 1. happen.

In [13]:
df[df.text.str.contains('^Chapter 1 ')].index.tolist()

[5, 22]

Remove the text up until Chapter 1 begins (there are 2 occurances)

In [14]:
df.drop(df.index[:df[df.text.str.contains('^Chapter 1 ')].index.tolist()[0]+1], inplace=True)
df.reset_index(inplace=True, drop=True)
df.drop(df.index[:df[df.text.str.contains('^Chapter 1 ')].index.tolist()[0]], inplace=True)
df.reset_index(inplace=True, drop=True)

Remove lines delineating volumes

Make a table of contents list with chapters

In [16]:
TOC = df[df.text.str.contains('^Chapter')]['text'].tolist()

In [17]:
len(TOC)

17

Combine chapter data together

In [18]:
corpus_dict = {}
i = 0
for k,v in df[1:].to_dict()['text'].items():
    if i not in corpus_dict:
        corpus_dict[i] = []
    if v not in TOC:
        corpus_dict[i].append(v)
    else:
        i += 1
        corpus_dict[i] = []

In [19]:
len(corpus_dict)

17

Parse text into sentences, store them in a dict as prepartion to move into a dataframe with columns of title, chapter and sentence.

In [20]:
from nltk.tokenize import sent_tokenize
sentence_dict = {}
sentence_dict['title'] = []
sentence_dict['chapter'] = []
sentence_dict['sentence'] = []
sentence_dict['author'] = []
for k,v in corpus_dict.items():
    for sentence in sent_tokenize(' '.join(v)):
        sentence_dict['title'].append(title)
        sentence_dict['chapter'].append(TOC[k])
        sentence_dict['sentence'].append(sentence)
        sentence_dict['author'].append(author)

Place the dict into a data frame so the data can be output to a csv easily.

In [21]:
import pandas as pd
final_df = pd.DataFrame.from_dict(sentence_dict)

In [26]:
final_df.sample(5)

,author,chapter,sentence,title
850,James Matthew Barrie,Chapter 4 THE FLIGHT,In the old days at home the Neverland had alwa...,PETER PAN
2510,James Matthew Barrie,Chapter 14 THE PIRATE SHIP,There was not a child on board the brig that n...,PETER PAN
2068,James Matthew Barrie,Chapter 11 WENDY'S STORY,Now I want you to consider the feelings of the...,PETER PAN
2181,James Matthew Barrie,Chapter 11 WENDY'S STORY,"""Get your things, Peter,"" she cried, shaking.",PETER PAN
2833,James Matthew Barrie,"Chapter 15 ""HOOK OR ME THIS TIME""","It said ""half-past one!""",PETER PAN


In [27]:
final_df.to_csv('./peter_pan.csv', index=False)